Demonstration/proof-of-concept code to take the output of the eScriptorium_chars notebook and then run a basic classifier on the results for a given letter.
Assumes the following:
- A folder `~/Image_data/escr_chars/{doc_id}`, where `doc_id` can be specified below but is intended to be the id (pk) of a given document on an instance of eScriptorium. This is intended to be the output of the eScriptorium_chars notebook but does not need to be.
- Subfolders named for the different characters, containing images of instances of the characters
- The image sizes are assumed to be approximately 100x100 pixels. If they are very different then you may want to change the size of the VGG input layer accordingly (specified in `vgg_inputsize` below).

Based heavily on demonstration code by Chahan Vidal-Gorene provided in the context of a course on Computational Palaeography jointly taught by the two of us in the Master Humanités numériques at Université PSL.

Peter Stokes, EPHE-PSL, March 2025

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import disable_interactive_logging, enable_interactive_logging
from tqdm import tqdm
import cv2

In [ ]:
vgg_inputsize = (112,112)

In [ ]:
def load_and_preprocess_images(folder_path, target_size=vgg_inputsize):
    model = VGG16(weights='imagenet', include_top=False)
    feature_list = []
    image_paths = []
    
    for file in tqdm(os.listdir(folder_path), desc="Preprocessing images"):
        img_path = os.path.join(folder_path, file)
        try:
            img = image.load_img(img_path, target_size=target_size)
            img_array = image.img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)
            
            features = model.predict(img_array).flatten()
            feature_list.append(features)
            image_paths.append(img_path)
        except Exception as e:
            # Faulty image; just ignore for now
            print(f"Error for image {file}: {e}")
            
    return np.array(feature_list), image_paths

def reduce_dimensionality(features, n_components=50):
    pca = PCA(n_components=n_components)
    features_pca = pca.fit_transform(features)
    return features_pca, pca

def cluster_images(features_pca, method='kmeans', n_clusters=5):
    if method == 'kmeans':
        model = KMeans(n_clusters=n_clusters, random_state=42)
    elif method == 'dbscan':
        model = DBSCAN(eps=5, min_samples=2)
    
    labels = model.fit_predict(features_pca)
    return labels

def visualize_clusters(image_paths, labels):
    unique_labels = np.unique(labels)
    for label in unique_labels:
        idxs = np.where(labels == label)[0]
        plt.figure(figsize=(15, 5))
        for i, idx in enumerate(idxs[:60]):  # Show max 10 images per cluster
            img = cv2.imread(image_paths[idx])
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.subplot(5, 12, i + 1)
            plt.imshow(img)
            plt.axis('off')
        plt.suptitle(f'Cluster {label}')
        plt.show()

In [ ]:
# The default interactive logging displays one status bar per image which is much more annoying than helpful
#doc_id = 4982
doc_id = 2917
char = 'a'

folder_path = os.path.join(os.path.expanduser('~'), 'Image_data', 'escr_chars', str(doc_id), char)

disable_interactive_logging()
features, image_paths = load_and_preprocess_images(folder_path)
enable_interactive_logging()

In [ ]:
features_pca, pca = reduce_dimensionality(features)

In [ ]:
labels = cluster_images(features_pca, method='kmeans', n_clusters=3)

In [ ]:
visualize_clusters(image_paths, labels)

In [ ]:
def display_feature_map(image_path, target_size=vgg_inputsize):
    model = VGG16(weights='imagenet', include_top=False)
    
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    
    features = model.predict(img_array)
    heatmap = np.mean(features[0], axis=-1)
    heatmap = cv2.resize(heatmap, (target_size[1], target_size[0]))
    heatmap = np.uint8(255 * heatmap / np.max(heatmap))
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    img = cv2.imread(image_path)
    img = cv2.resize(img, target_size)
    superimposed_img = cv2.addWeighted(img, 0.6, heatmap, 0.4, 0)
    
    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title("Feature Activation Map")
    plt.show()

In [ ]:
display_feature_map(os.path.join(folder_path, 'char-2917-545034-0.png'))